In [1]:
input_filename_test = "input-example.txt"
input_filename = "input.txt"

In [2]:
MAX_SIGNAL = 65535
OPERATORS = {"AND", "OR", "NOT", "LSHIFT", "RSHIFT"}


class Instruction:
    def __init__(self, raw_line: str) -> None:
        self.orig = raw_line.strip()
        
        raw_instruction, wire = self.orig.split(" -> ")
        self.wire = wire
        self.input_wires = []
        self.parts = []
        for part in raw_instruction.split():
            try:
                self.parts.append(int(part))
            except ValueError:
                self.parts.append(part)
                if part not in OPERATORS:
                    self.input_wires.append(part)
        
        # We expect only 1, 2, or 3 components before the "->"
        if len(self.parts) > 3 or len(self.parts) < 1:
            raise ValueError("Unexpected instruction format:", raw_instruction)

    def __repr__(self) -> str:
        return f"Instruction({self.orig})"
    
    def __str__(self) -> str:
        return self.orig
            
    def update(self, wire: str, signal: int) -> None:
        for i, part in enumerate(self.parts):
            if part == wire:
                self.parts[i] = signal
                
    def can_reduce(self) -> bool:
        if len(self.parts) == 1:
            if isinstance(self.parts[0], int):
                return True
            
        elif len(self.parts) == 2:
            if isinstance(self.parts[1], int):
                return True
            
        elif len(self.parts) == 3:
            if isinstance(self.parts[0], int) and isinstance(self.parts[2], int):
                return True
        
        return False
                
    def reduce(self) -> int:
        """
        Produce the signal value for this instruction.
        Can only reduce if all input wires are replaced with their signal values.
        """
        
        if len(self.parts) == 1:
            assert isinstance(self.parts[0], int)
            return int(self.parts[0])
        
        # Unary operations (it's only NOT)
        elif len(self.parts) == 2:
            assert isinstance(self.parts[1], int)
            if self.parts[0] != "NOT":
                raise ValueError("Unexpected unary operator:", self.parts[0])
            
            return MAX_SIGNAL - self.parts[1]
        
        # Binary operations
        elif len(self.parts) == 3:
            assert isinstance(self.parts[0], int)
            assert isinstance(self.parts[2], int)
            
            operator = self.parts[1]
            if operator == "AND":
                return self.parts[0] & self.parts[2]
            elif operator == "OR":
                return self.parts[0] | self.parts[2]
            elif operator == "LSHIFT":
                return self.parts[0] << self.parts[2]
            elif operator == "RSHIFT":
                return self.parts[0] >> self.parts[2]
            
            else:
                raise ValueError("Unexpected binary operator:", operator)
                
        
        else:
            raise ValueError("Unexpected number of instruction parts:", self.parts)

In [3]:
from collections import defaultdict
from typing import Dict


def process_circuit(input_filename: str, is_part_2: bool = False) -> Dict[str, int]:
    with open(input_filename) as input_file:
        booklet_lines = input_file.readlines()

    wire_to_signal = {}  # wire name to signal value
    wire_to_instructions = defaultdict(list)  # wire to instructions where it is an input

    # Initial processing of instructions
    stack = []
    for line in booklet_lines:
        instruction = Instruction(line)
        
        # lazy way to get part 2 answer
        if is_part_2:
            if instruction.wire == "b":
                instruction.parts[0] = 956
        
        if instruction.can_reduce():
            wire_to_signal[instruction.wire] = instruction.reduce()
            stack.append(instruction.wire)

        for input_wire in instruction.input_wires:
            wire_to_instructions[input_wire].append(instruction)
            
    # Replace wire names with signal values in instructions until they're all replaced
    while stack:
        curr_wire = stack.pop()
        curr_signal = wire_to_signal[curr_wire]
        for instruction in wire_to_instructions[curr_wire]:
            instruction.update(curr_wire, curr_signal)

            if instruction.can_reduce():
                # update our data structures
                wire_to_signal[instruction.wire] =  instruction.reduce()
                stack.append(instruction.wire)
                
    return wire_to_signal

# Part 1

In [4]:
# Test case
assert process_circuit(input_filename_test) == {
    "d": 72,
    "e": 507,
    "f": 492,
    "g": 114,
    "h": 65412,
    "i": 65079,
    "x": 123,
    "y": 456,
}

In [5]:
# Part 1 answer
process_circuit(input_filename)["a"]

956

# Part 2

In [6]:
process_circuit(input_filename, is_part_2=True)["a"]

40149